In [1]:
from google.colab import files
uploaded = files.upload()

Saving data.zip to data.zip


In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/.ipynb_checkpoints/
  inflating: data/.ipynb_checkpoints/Untitled-checkpoint.ipynb  
   creating: data/test/
  inflating: data/test/IM-0001-0001.jpeg  
  inflating: data/test/IM-0003-0001.jpeg  
  inflating: data/test/IM-0005-0001.jpeg  
  inflating: data/test/IM-0006-0001.jpeg  
  inflating: data/test/IM-0007-0001.jpeg  
  inflating: data/test/IM-0009-0001.jpeg  
  inflating: data/test/IM-0010-0001.jpeg  
  inflating: data/test/IM-0011-0001-0001.jpeg  
  inflating: data/test/person100_bacteria_475.jpeg  
  inflating: data/test/person100_bacteria_477.jpeg  
  inflating: data/test/person100_bacteria_478.jpeg  
  inflating: data/test/person100_bacteria_479.jpeg  
  inflating: data/test/person100_bacteria_480.jpeg  
  inflating: data/test/person100_bacteria_481.jpeg  
  inflating: data/test/person100_bacteria_482.jpeg  
  inflating: data/test/person101_bacteria_483.jpeg  
   creating: data/train/
   creating: data/train/Normal/
  inflating: data/train/

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalize images: normalization typically entails scaling values to between 0 and 1.
                  # Images are made of pixels with varying intensities from 0 to 255.
                  # So normalizing the images (dividing each pixel by 255) scales the pixel values to between 0 and 1.
                  # This range of values is more effective for a neural network to learn from.
train_data_generator = ImageDataGenerator(rescale=1/255)
validate_data_generator = ImageDataGenerator(rescale=1./255)

# Indicate the location of our train and validation set of images.
train_folder = "data/train/"
validation_folder = "data/Validate/"

# Provide images to the model in batches using the train_data_generator
train_generator = train_data_generator.flow_from_directory(
        train_folder,  # This indicates where the images are located.
                        # It contains folders whose names will be the class labels of the enclosed images.
        target_size=(150, 150), # All images will be resized to 150x150
        batch_size=10,        # The generator will provide the model with 10 images at a time as it's "learning."
        class_mode="binary")   # We use "binary" because there are two classes (i.e. normal, hemorrhage)

# Provide the model with a validation set of images using the validate_data_generator
validate_generator = validate_data_generator.flow_from_directory(
        validation_folder,
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=10,
        class_mode='binary')

Found 32 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [4]:
from tensorflow.keras import models, layers

model = models.Sequential([

    ### DEEP LEARNING ###

    # Layer 1
    # Convolution (learning 16 filters) & Pooling
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), # The shape of the images is 150x150.
    layers.MaxPooling2D(),                       # The depth of the image is 1 since the image is only grayscale.

    # Layer 2
    # Convolution (learning 32 filters) & Pooling Layer
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    # Layer 3
    # Convolution (learning 64 filters) & Pooling Layer
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    # Layer 4
    # Convolution (learning 64 filters) & Pooling Layer
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    # Layer 5
    # Convolution (learning 128 filters) & Pooling Layer
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),


    # Fully-Connected Layer
    # Flatten the learned features into a 1-dimensional array of values.
    # Pass the learned features to the fully-connected layer (Dense layer in TensorFlow) for prediction.
    layers.Flatten(),
    layers.Dense(256, activation='relu'), # 'relu' helps the model focus on what's most important and speeds up training
    layers.Dense(1, activation='sigmoid') # 'sigmoid' ensures that the prediction will be between 0 and 1,
                                          # interpreted as the probability of being in the positive class ("normal").
])



# Compile the Built Model
# 'binary_crossentropy' (this is a binary task) tells the model how to measure its loss (error)
# 'adam' indicates how the model will adjust its weights, while learning, to decrease its loss (error)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = ['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
history = model.fit(
      train_generator,  # Provides the training images to the model.
      validation_data = validate_generator, # Provides the validation set of images to the model to assess training.
      epochs=10)        # The number of times the model will get to go through all 140 of the training images.

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 352ms/step - accuracy: 0.4850 - loss: 0.7101 - val_accuracy: 0.5000 - val_loss: 0.7164
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 352ms/step - accuracy: 0.4258 - loss: 0.7307 - val_accuracy: 0.5000 - val_loss: 0.6948
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 457ms/step - accuracy: 0.6333 - loss: 0.6862 - val_accuracy: 0.5000 - val_loss: 0.6964
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 416ms/step - accuracy: 0.4709 - loss: 0.6915 - val_accuracy: 0.5000 - val_loss: 0.6945
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step - accuracy: 0.5492 - loss: 0.6872 - val_accuracy: 0.4000 - val_loss: 0.6939
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 351ms/step - accuracy: 0.8004 - loss: 0.6896 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 267ms/step - accuracy: 0.6072 - loss: 0.6878 - val_accuracy: 0.4000 - val_loss: 0.6939
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 270ms/step - accuracy: 0.5658 - loss: 0.6807 - val_accuracy: 0.5000 - val_loss:

In [6]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os

#  The location of our test (unseen) images, used to evaluate the model's predictions
test_images = os.listdir("data/test")

# Make predictions on the test images
for file_name in test_images:
    path = "data/test/" + file_name
    test_img = image.load_img(path, target_size = (150,150)) # Resize images to the same size the model trained on.
    img = image.img_to_array(test_img)
    img /= 255.0    # Normalize the images (scale the values to between 0 and 1).
    img = np.expand_dims(img, axis = 0)

  # Predict an image's class
    prediction = model.predict(img)

  # Output the prediction score
    print(f"Prediction: {prediction[0]}") # The model's probability score that the image is "normal"
                                          # Probability < 0.5 predicts "hemorrhage".
                                          # Probability >= 0.5 predicts "normal".
  # Output the file name
    if prediction[0] < 0.5:
        print(file_name + " is pneumonia")

    else:
        print(file_name + " is normal")
    print('\n')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Prediction: [0.4487171]
person100_bacteria_481.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Prediction: [0.47218907]
person101_bacteria_483.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Prediction: [0.45553577]
IM-0009-0001.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: [0.43931916]
IM-0011-0001-0001.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: [0.44656175]
IM-0007-0001.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: [0.45012975]
person100_bacteria_478.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: [0.45880204]
person100_bacteria_477.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: [0.45748645]
person100_bacteria_480.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction: [0.4276029]
person100_bacteria_475.jpeg is pneumonia


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Prediction: [0.46245387]
person100